<a href="https://colab.research.google.com/github/ga642381/ML2021-Spring/blob/main/HW02/HW02-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework 2-1 Phoneme Classification**

* Slides: https://speech.ee.ntu.edu.tw/~hylee/ml/ml2021-course-data/hw/HW02/HW02.pdf
* Video (Chinese): https://youtu.be/PdjXnQbu2zo
* Video (English): https://youtu.be/ESRr-VCykBs


## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [131]:
import numpy as np

print('Loading data ...')

data_root='./ml2021spring-hw2/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


## Create Dataset

In [132]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None, tr_mean=None, tr_std=None):
        self.data = torch.from_numpy(X).float()
        # if tr_mean is not None and tr_std is not None:

        #     self.mean = self.data.mean(dim = 0,keepdim=True)
        #     self.std = self.data.std(dim = 0,keepdim=True)
            
        #     self.data = (self.data - self.mean) / self.std
        # else:
            
        #     self.mean = self.data.mean(dim = 0,keepdim=True)
        #     self.std = self.data.std(dim = 0,keepdim=True)

        #     self.data = (self.data - self.mean) / self.std
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)
    
    def normalize(self):
        return self.mean, self.std
    

Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [133]:
import random

VAL_RATIO = 0.2
percent = int(train.shape[0] * (1 - VAL_RATIO))

val_of_dataset = []
tr_of_dataset = []
for i in range(len(train)):
    x = random.randint(1, 6)
    if x % 5 == 0:
        val_of_dataset.append(i)
    else:
        tr_of_dataset.append(i)
# train[:,195:234] = train[:,195:234]*5
train_x, train_y, val_x, val_y = train[tr_of_dataset,:], train_label[tr_of_dataset], train[val_of_dataset,:], train_label[val_of_dataset]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (1024953, 429)
Size of validation set: (204979, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [134]:
BATCH_SIZE = 640

from torch.utils.data import DataLoader
tr_mean = None
tr_std = None
train_set = TIMITDataset(train_x, train_y,None,None)
# tr_mean,tr_std = train_set.normalize()
val_set = TIMITDataset(val_x, val_y,tr_mean,tr_std)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

C:\Users\78739\AppData\Local\Temp\ipykernel_22764\3353645887.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = y.astype(np.int)


Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [135]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

41

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [136]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self,inDim=195):
        super(Classifier, self).__init__()

        self.layer1 = nn.Linear(inDim, 2048)
        self.layer2 = nn.Linear(2048, 2048)
        self.layer3 = nn.Linear(2048, 1024)
        self.layer4 = nn.Linear(1024, 512)
        self.layer5 = nn.Linear(512, 128)
        self.out = nn.Linear(128, 39) 

        self.bn1 = nn.BatchNorm1d(2048)
        self.bn2 = nn.BatchNorm1d(2048)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(128)
        self.bn6 = nn.BatchNorm1d(64)

        self.drop1 = nn.Dropout(0.3)
        # self.drop2 = nn.Dropout(0.3)

        self.act_re = nn.ReLU()
        self.act_sig = nn.Sigmoid()

    def forward(self, x):
        
        x = self.layer1(x)
        x = self.bn1(x)
        x = self.act_re(x)
        x = self.drop1(x)

        x = self.layer2(x)
        x = self.bn2(x)
        x = self.act_re(x)
        x = self.drop1(x)

        x = self.layer3(x)
        x = self.bn3(x)
        x = self.act_re(x)
        x = self.drop1(x)

        x = self.layer4(x)
        x = self.bn4(x)
        x = self.act_re(x)
        x = self.drop1(x)

        x = self.layer5(x)
        x = self.bn5(x)
        x = self.act_re(x)
        x = self.drop1(x)

        # x = self.layer6(x)
        # x = self.bn6(x)
        # x = self.act_re(x)
        # x = self.drop1(x)



        x = self.out(x)

        return x

    def reg_l2(self):
        l2 = 0
        for param in self.parameters():
            l2 += l2_lambda*torch.sum(param**2)
        return l2

## Training

In [137]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [138]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [139]:
# fix random seed for reproducibility
same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 30               # number of training epoch
learning_rate = 0.00007       # learning rate
l2_lambda = 0            # weight decay
inDim = 429       # input dimension
# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer, 
model = Classifier(inDim).to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


DEVICE: cuda


In [140]:
# start training

best_acc = 0.0

for epoch in range(num_epoch):
    
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)+model.reg_l2()
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        optimizer.zero_grad() 
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels)+model.reg_l2()
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


[001/030] Train Acc: 0.511411 Loss: 1.825125 | Val Acc: 0.637451 loss: 1.211447
saving model with acc 0.637
[002/030] Train Acc: 0.620335 Loss: 1.283984 | Val Acc: 0.681948 loss: 1.027773
saving model with acc 0.682
[003/030] Train Acc: 0.649134 Loss: 1.161926 | Val Acc: 0.699755 loss: 0.955044
saving model with acc 0.700
[004/030] Train Acc: 0.665560 Loss: 1.096022 | Val Acc: 0.712492 loss: 0.908464
saving model with acc 0.712
[005/030] Train Acc: 0.676942 Loss: 1.050737 | Val Acc: 0.720835 loss: 0.877178
saving model with acc 0.721
[006/030] Train Acc: 0.685778 Loss: 1.015521 | Val Acc: 0.727518 loss: 0.849604
saving model with acc 0.728
[007/030] Train Acc: 0.693546 Loss: 0.985608 | Val Acc: 0.734300 loss: 0.823385
saving model with acc 0.734
[008/030] Train Acc: 0.700180 Loss: 0.959872 | Val Acc: 0.738363 loss: 0.806450
saving model with acc 0.738
[009/030] Train Acc: 0.706445 Loss: 0.936390 | Val Acc: 0.744130 loss: 0.787929
saving model with acc 0.744
[010/030] Train Acc: 0.71168

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [141]:
# create testing dataset

test_set = TIMITDataset(test, None,tr_mean, tr_std)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier(inDim).to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [142]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [143]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))